# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data/")

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id', 'title','artist_id', 'year', 'duration']].values[0].tolist()
song_data = (song_data[0], song_data[1], song_data[2], song_data[3], song_data[4])
song_data

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
song_table_insert
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data = (artist_data[0], artist_data[1], artist_data[2], artist_data[3], artist_data[4])
artist_data

('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
artist_table_insert
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files("data/log_data/")

In [12]:
filepath = log_files[0]

In [13]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Let's Live For Today,200,1542153802796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,1542153968796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
2,Eddie Palmieri,Logged In,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Nada De Ti,200,1542154266796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
3,The Bravery,Logged In,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Give In,200,1542154657796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
4,K.U.K.L,Logged In,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Anna,200,1542154825796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df['page']=="NextSong"].reset_index()
df.head()

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Let's Live For Today,200,1542153802796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
1,1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,1542153968796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
2,2,Eddie Palmieri,Logged In,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Nada De Ti,200,1542154266796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
3,3,The Bravery,Logged In,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Give In,200,1542154657796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
4,4,K.U.K.L,Logged In,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Anna,200,1542154825796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


In [15]:
import datetime
t = pd.to_datetime(df.ts, unit='ms')
t.head()

0   2018-11-14 00:03:22.796
1   2018-11-14 00:06:08.796
2   2018-11-14 00:11:06.796
3   2018-11-14 00:17:37.796
4   2018-11-14 00:20:25.796
Name: ts, dtype: datetime64[ns]

In [16]:
df['week'] = t.apply(lambda x: datetime.date(x.year, x.month, x.day).isocalendar()[1])
df['week_day'] = t.apply(lambda x: datetime.date(x.year, x.month, x.day).strftime("%A"))
df.head()

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,...,page,registration,sessionId,song,status,ts,userAgent,userId,week,week_day
0,0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",...,NextSong,1.540809e+12,411,Let's Live For Today,200,1542153802796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,46,Wednesday
1,1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",...,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,1542153968796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,46,Wednesday
2,2,Eddie Palmieri,Logged In,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",...,NextSong,1.540809e+12,411,Nada De Ti,200,1542154266796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,46,Wednesday
3,3,The Bravery,Logged In,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",...,NextSong,1.540809e+12,411,Give In,200,1542154657796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,46,Wednesday
4,4,K.U.K.L,Logged In,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",...,NextSong,1.540809e+12,411,Anna,200,1542154825796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,46,Wednesday


In [17]:
time_data = (t, t.dt.hour, t.dt.day, df.week, t.dt.month, t.dt.year, df.week_day)
column_labels = ['start_time','hour','day','week','month', 'year','weekday']

In [18]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()
df['start_time'] = t
df.head()

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,...,registration,sessionId,song,status,ts,userAgent,userId,week,week_day,start_time
0,0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",...,1.540809e+12,411,Let's Live For Today,200,1542153802796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,46,Wednesday,2018-11-14 00:03:22.796
1,1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",...,1.540809e+12,411,Time Can Never Kill The True Heart,200,1542153968796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,46,Wednesday,2018-11-14 00:06:08.796
2,2,Eddie Palmieri,Logged In,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",...,1.540809e+12,411,Nada De Ti,200,1542154266796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,46,Wednesday,2018-11-14 00:11:06.796
3,3,The Bravery,Logged In,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",...,1.540809e+12,411,Give In,200,1542154657796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,46,Wednesday,2018-11-14 00:17:37.796
4,4,K.U.K.L,Logged In,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",...,1.540809e+12,411,Anna,200,1542154825796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,46,Wednesday,2018-11-14 00:20:25.796


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [19]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [20]:
user_df =  df[['userId','firstName', 'lastName', 'gender', 'level']]
user_df.head()
user_table_insert

'INSERT INTO users(user_id, first_name, last_name, gender,level)\n                        VALUES(%s, %s, %s, %s, %s)\n'

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [21]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()
    

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [22]:
song_select

'SELECT songs.song_id, artists.artist_id FROM songs \n                  JOIN artists ON  songs.artist_id=artists.artist_id;\n'

In [23]:
songplay_table_insert

'INSERT INTO songplays( start_time, user_id,level,artist_id,song_id, session_id, location, user_agent)\nVALUES(%s, %s, %s, %s, %s, %s, %s, %s)\n'

In [24]:
for index, row in df.iterrows():
   
       # get songid and artistid from song and artist tables
       cur.execute(song_select)
       results = cur.fetchone()
       print(results)
       
       if results:
            songid, artistid = results
       else:
            songid, artistid = None, None
   
        # insert songplay record
       print(songplay_table_insert)
       songplay_data = (pd.to_datetime(row.ts, unit='ms'),row.userId,row.level,artistid,songid,row.sessionId,row.location,row.userAgent)
       print(songplay_data)
       cur.execute(songplay_table_insert, songplay_data)
       conn.commit()

('SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1')
INSERT INTO songplays( start_time, user_id,level,artist_id,song_id, session_id, location, user_agent)
VALUES(%s, %s, %s, %s, %s, %s, %s, %s)

(Timestamp('2018-11-14 00:03:22.796000'), '95', 'paid', 'ARD7TVE1187B99BFB1', 'SOMZWCG12A8C13C480', 411, 'Winston-Salem, NC', '"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"')
('SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1')
INSERT INTO songplays( start_time, user_id,level,artist_id,song_id, session_id, location, user_agent)
VALUES(%s, %s, %s, %s, %s, %s, %s, %s)

(Timestamp('2018-11-14 00:06:08.796000'), '95', 'paid', 'ARD7TVE1187B99BFB1', 'SOMZWCG12A8C13C480', 411, 'Winston-Salem, NC', '"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"')
('SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1')
INSERT INTO songplays( start_time, user_id,

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [25]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import datetime
from sql_queries import *

def process_song_file(cur, filepath):
    # open song file
    df =pd.read_json(filepath, lines=True)
        
    # insert song record
    song_data = df[['song_id', 'title','artist_id', 'year', 'duration']].values[0].tolist()
    song_data = (song_data[0], song_data[1], song_data[2], song_data[3], song_data[4])
    cur.execute(song_table_insert, song_data)
    
    
    # insert artist record

    artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
    artist_data = (artist_data[0], artist_data[1], artist_data[2], artist_data[3], artist_data[4])
    cur.execute(artist_table_insert, artist_data)
    


def process_log_file(cur, filepath):
    # open log file
    df = pd.read_json(filepath, lines=True)

    # filter by NextSong action
    df = df[df['page']=="NextSong"].reset_index() 
    # convert timestamp column to datetime
    t = pd.to_datetime(df.ts, unit='ms')
    df['week'] = t.apply(lambda x: datetime.date(x.year, x.month, x.day).isocalendar()[1])
    df['week_day'] = t.apply(lambda x: datetime.date(x.year, x.month, x.day).strftime("%A"))
    
    # insert time data records
    time_data = (t, t.dt.hour, t.dt.day, df.week, t.dt.month, t.dt.year, df.week_day)
    column_labels = ['start_time','hour','day','week','month', 'year','weekday']
    time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
    df['start_time'] = t
    
    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))
    
    # load user table
    user_df = df[['userId','firstName', 'lastName', 'gender', 'level']] 
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, row)
    
    # insert user records
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, row)

    # insert songplay records
    for index, row in df.iterrows():
        
        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
        
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None

        # insert songplay record

    for index, row in df.iterrows():
   
       # get songid and artistid from song and artist tables
       cur.execute(song_select)
       results = cur.fetchone()

       if results:
            songid, artistid = results
       else:
            songid, artistid = None, None

       songplay_data = (pd.to_datetime(row.ts, unit='ms'),row.userId,row.level,artistid,songid,row.sessionId,row.location,row.userAgent)
       cur.execute(songplay_table_insert, songplay_data)
       


def process_data(cur, conn, filepath, func):
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))

    # iterate over files and process
    for i, datafile in enumerate(all_files, 1):
        func(cur, datafile)
        conn.commit()
        print('{}/{} files processed.'.format(i, num_files))


def main():
    
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur = conn.cursor()

    process_data(cur, conn, filepath='data/song_data', func=process_song_file)
    process_data(cur, conn, filepath='data/log_data', func=process_log_file)

    conn.close()


if __name__ == "__main__":
    main()

71 files found in data/song_data
1/71 files processed.
2/71 files processed.
3/71 files processed.
4/71 files processed.
5/71 files processed.
6/71 files processed.
7/71 files processed.
8/71 files processed.
9/71 files processed.
10/71 files processed.
11/71 files processed.
12/71 files processed.
13/71 files processed.
14/71 files processed.
15/71 files processed.
16/71 files processed.
17/71 files processed.
18/71 files processed.
19/71 files processed.
20/71 files processed.
21/71 files processed.
22/71 files processed.
23/71 files processed.
24/71 files processed.
25/71 files processed.
26/71 files processed.
27/71 files processed.
28/71 files processed.
29/71 files processed.
30/71 files processed.
31/71 files processed.
32/71 files processed.
33/71 files processed.
34/71 files processed.
35/71 files processed.
36/71 files processed.
37/71 files processed.
38/71 files processed.
39/71 files processed.
40/71 files processed.
41/71 files processed.
42/71 files processed.
43/71 file